In [19]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
from random import randint
from collections import OrderedDict 
import numpy as np
import pandas as pd
import random
import sys
import io
import re
from unidecode import unidecode

In [20]:
songs = pd.read_csv('data/wu-tang-clan-lyrics.csv')

In [21]:
def get_tokenized_lines(df):
    words = []
    
    for index, row in df['lyrics'].iteritems():
        row = str(row).lower()
        for line in row.split('|-|'):
            new_words = re.findall(r"\b[a-z']+\b", unidecode(line))
            words = words + new_words
        
    return words

In [22]:
all_lyric_lines = get_tokenized_lines(songs)

In [23]:
SEQ_LENGTH = 50 + 1
sequences = list()

for i in range(SEQ_LENGTH, len(all_lyric_lines)):
    seq = all_lyric_lines[i - SEQ_LENGTH: i]
    sequences.append(seq)

print('Total Sequences: %d' % len(sequences))

Total Sequences: 18887


In [24]:
def save_doc(lines, filename):
    for line in lines:
        data = ' '.join(line)
        
    '\n'.join(data)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [25]:
out_filename = 'sequences.txt'
save_doc(sequences, out_filename)

In [26]:
vocab = set(all_lyric_lines)
vocab = sorted(vocab)

word_to_index = {w: i for i, w in enumerate(vocab)}
index_to_word = {i: w for w, i in word_to_index.items()}
word_indices = [word_to_index[word] for word in vocab]
vocab_size = len(vocab)

print(word_to_index)
print('vocabulary size: {}'.format(vocab_size))

{"'s": 0, 'a': 1, 'aa': 2, 'aah': 3, 'abbot': 4, 'abbott': 5, 'abel': 6, 'abilities': 7, 'able': 8, 'about': 9, 'above': 10, 'absolut': 11, "ac's": 12, 'academically': 13, 'acapella': 14, 'accepted': 15, 'ace': 16, 'achie': 17, 'across': 18, 'act': 19, 'actin': 20, 'acts': 21, 'actual': 22, 'add': 23, 'adding': 24, 'addition': 25, 'adidas': 26, 'adina': 27, 'adjourned': 28, 'administration': 29, 'admire': 30, 'advantage': 31, 'adventure': 32, 'adviser': 33, 'afraid': 34, 'african': 35, 'afro': 36, 'after': 37, 'again': 38, 'against': 39, 'age': 40, 'ageless': 41, 'agenda': 42, 'agent': 43, 'ago': 44, 'agony': 45, 'ah': 46, 'ahead': 47, 'ahh': 48, 'ahhh': 49, 'aight': 50, 'aiii': 51, 'aim': 52, "ain't": 53, 'aint': 54, 'airin': 55, 'airplane': 56, 'airtime': 57, 'airwaves': 58, 'aisle': 59, 'aiyo': 60, 'aiyy': 61, 'aiyyo': 62, 'ak': 63, 'akh': 64, 'al': 65, 'alabama': 66, 'albert': 67, 'alex': 68, 'ali': 69, 'alive': 70, 'all': 71, 'allah': 72, "allah's": 73, 'allow': 74, 'almost': 75, 

In [27]:
def get_tokenized_lines(lines, seq_len):
    tokenized = np.zeros((len(lines), seq_len))
    
    for r, line in enumerate(lines):
        for c, word in enumerate(line):
            tokenized[r, c] = word_to_index[word]

    return tokenized

In [28]:
tokenized_seq = get_tokenized_lines(sequences, SEQ_LENGTH)

In [29]:
tokenized_seq[:, -1].shape

(18887,)

In [30]:
from keras.utils import to_categorical

X, y = tokenized_seq[:, :-1], tokenized_seq[:, -1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = len(X[0])

In [31]:
print("X_shape", X.shape)
print("y_shape", y.shape)

X_shape (18887, 50)
y_shape (18887, 4069)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X, y, batch_size=128, epochs=200)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 50)            203450    
_________________________________________________________________
lstm_4 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 4069)              410969    
Total params: 765,319
Trainable params: 765,319
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
148/148 [==============================] - 11s 75ms/step - loss: 7.2257 - accuracy: 0.0507
Epoch 

In [ ]:
model.save('word_model_wu_tang_clan.h5')

In [56]:
from keras.models import load_model

model = load_model('word_model_eminem.h5')

In [57]:
def texts_to_sequences(texts, word_to_index):
    indices = np.zeros((1, len(texts)), dtype=int)
    
    for i, text in enumerate(texts):
        if text not in word_to_index:
            random = index_to_word[randint(0,vocab_size)]
            indices[:, i] = word_to_index[random]
        else:
            indices[:, i] = word_to_index[text]
        
    return indices

In [58]:
def my_pad_sequences(seq, maxlen):
    start = seq.shape[1] - maxlen
    
    return seq[:, start: start + maxlen]

In [59]:
def generate_seq(model, word_to_index, seq_length, n_words):
    generated = ''
    usr_input = input("Write the beginning of your poem, the Drake machine will complete it. Your input is: ")
    generated += usr_input 
    sys.stdout.write("\n\nHere is your poem: \n\n") 
    
    result = list()
    in_text = [None] * 51
    generated_list = generated.split()
    
    # if input is shorter than 51 words, fill the beginning with random words
    if(len(generated_list) < 51):
        end = len(generated_list)
        for i in range (51 - end):
            random = index_to_word[randint(0,vocab_size)]
            in_text[i] = random
            
        index = 0
        for i in range (51 - end, 51):
            in_text[i] = generated_list[index]
            index += 1

    # if input is longer than 51 words, only use the last 51 words
    if(len(generated_list) > 51):
        end = len(generated_list)
        in_text = generated_list[end-51:]

    # generate words based on last 50 words
    for _ in range(n_words):
        encoded = texts_to_sequences(in_text[1:], word_to_index)
        encoded = my_pad_sequences(encoded, maxlen=seq_length)
        
        yhat = model.predict_classes(encoded, verbose=0)
        out_word = ''
    
        for word, index in word_to_index.items():
            if index == yhat:
                out_word = word
                break
        
        in_text += ' ' + out_word
        result.append(out_word)
        
    return ' '.join(result)

In [60]:
generated = generate_seq(model, word_to_index, seq_length, 50)
print(generated)

Write the beginning of your poem, the Drake machine will complete it. Your input is: sdahfjashdfkjashd asdfh


Here is your poem: 

['coke', 'cookin', "problem's", 'headache', 'fifth', 'ovulating', 'birth', 'dear', 'neither', 'grips', 'aight', 'pacquiao', 'truthful', 'mann', 'babe', 'misconducts', 'extraterrestrial', 'dodgers', 'smack', 'ruin', 'jade', 'chasing', 'cars', 'poppins', 'tear', 'legendary', 'sorry', 'plenty', 'lewinsky', 'outdo', 'raps', 'growing', 'wide', 'sister', "you'ont", 'opposites', 'winks', 'glide', 'huge', 'southpaw', 'grasp', 'symbolic', 'harris', 'come', 'helped', 'muster', 'sit', 'beer', 'sweden', 'sdahfjashdfkjashd', 'asdfh']
envision fellatio fellow fellatio fellow denver clothes flippity christmas flippity cat triangle uh sa minimum fellatio fellow fellatio fellow fellatio groups resentment lazy hangs pectations jump pigeonholed ho's pinkett clothes pinkett clothes pinkett clothes pinkett lazy hangs bury idaho west pigeonholed fellatio fellow fellatio fellow 